In [1]:
import numpy as np

$$Mkn=e^(\frac{−i*2π*k*n}{N})$$

In [3]:
def DFT(x): #slow: O(N^2)
    x = np.asarray(x, dtype = float)
    N = x.shape[0] #1D
    
    n = np.arrange(N)
    k = n.reshape(N, 1)
    
    M = np.exp(-j * 2 * np.pi * k * n / N)    

In [9]:
def FFT(x): #fast: O(Nlog(N))
    N = x.length
    half = N/2
    
    #even
    even = np.zeros(half,dtype=np.complex_)
    for i in range(half):
        even[i] = x[2*i]
    even_part = FFT(even)
    
    #odd
    odd = even
    for i in range(half):
        odd[i] = x[2*i + 1]
    odd_part = FFT(odd)
    
    #merging
    final = np.zeros(half,dtype=np.complex_)
    for i in range(half):
        power = -2 * i * np.pi / N
        w_power = np.cos(power) + j*np.sin(power)
        final[i] = even_part[i] + w_power * odd_part[i]
        final[i + half] = even_part[i] - w_power * odd_part[i]
    
    return final